In [1]:

## ALL IMPORTS FOR A NEW NOTEBOOK

import os, sys, random, math
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
import seaborn as sns
import itertools as it
import scipy
import glob
import matplotlib
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset,DataLoader
from torch.optim import Optimizer
import torchvision.transforms.transforms as txf
import torch.optim.lr_scheduler as lr_scheduler
from collections import OrderedDict

from sklearn import metrics
from sklearn import preprocessing as pp
from sklearn import model_selection as ms

import torch_utils
from tqdm.notebook import tqdm_notebook as tqdm
import time

font = {'size'   : 20}

matplotlib.rc('font', **font)


In [2]:
from torchtext import data, datasets
SEED = 947
torch_utils.seed_everything(SEED)

In [35]:
TEXT = data.Field(tokenize="spacy")
LABEL = data.LabelField()

In [36]:
%%time
train_data, test_data = datasets.TREC.splits(TEXT, LABEL, fine_grained=False)
train_data, valid_data = train_data.split(split_ratio=0.8, random_state=random.seed(SEED))

CPU times: user 1.66 s, sys: 63.7 ms, total: 1.72 s
Wall time: 760 ms


In [37]:
len(train_data), len(valid_data), len(test_data)

(4362, 1090, 500)

In [38]:
vars(train_data[5])

{'text': ['What', 'are', 'Bellworts', '?'], 'label': 'DESC'}

In [39]:
MAX_VOCAB_SIZE = 25000
TEXT.build_vocab(
    train_data,
    max_size=MAX_VOCAB_SIZE,
    vectors="glove.6B.100d",
    unk_init=torch.Tensor.normal_
)

LABEL.build_vocab(train_data)

In [40]:
LABEL.vocab.stoi

defaultdict(None,
            {'ENTY': 0, 'HUM': 1, 'DESC': 2, 'NUM': 3, 'LOC': 4, 'ABBR': 5})

In [41]:
BATCH_SIZE = 64
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data),
    batch_sizes=(BATCH_SIZE, BATCH_SIZE, BATCH_SIZE),
    device=device
)